In [1]:
from solid import translate, rotate, union
from solid import cylinder, sphere
from solid import scad_render
from viewscad import Renderer

from datetime import datetime

import numpy as np

renderer = Renderer(openscad_exec='C:\Program Files\OpenSCAD\openscad.exe')

In [42]:
start_time = datetime.now()

# tree randomizer
%run Tree.py

t = Tree([0, 0, 0], [0, 0, 0])
t.grow(4)
tn = t.getParts()

print('estimate time:', len(tn) * 2, 's')

# renderable parts
renderer.render(t.getObject())

print(f'{(datetime.now() - start_time).total_seconds():0.3f}')

estimate time: 30 s


30.806


In [44]:
import pprint

config = t.getConfig()

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config)

[   [   [0, 0, 0],
        [0, -6.772092199518539, -70.06527309812998],
        [50.0, 2, 1.5],
        [2]],
    [   [-2.0102427080360736, 5.542735403481017, 49.65115314372654],
        [0, -14.273343284308194, 73.87597562043908],
        [37.5, 1.5, 1.0],
        [1.5]],
    [   [-4.5778952316320645, -3.339126002726272, 85.99354847588643],
        [0, -2.1121294287160346, 13.767033227087751],
        [25.0, 1.0, 0.5],
        [1.0]],
    [   [-5.472806801290242, -3.5583914657973184, 110.97656382510311],
        [0, -9.11533904639986, -126.3394034768938],
        [12.5, 0.5, 0.0],
        [0.5]],
    [   [-2.0102427080360736, 5.542735403481017, 49.65115314372654],
        [0, 62.563099336640576, 80.62450196431044],
        [37.5, 1.125, 0.75],
        [1.125]],
    [   [3.4115226324419363, 38.38010780565551, 66.93008350164222],
        [0, 71.207129013644, 92.5829115942785],
        [25.0, 0.75, 0.375],
        [0.75]],
    [   [2.3449576967363237, 62.0232969591614, 74.98378115486119]

In [ ]:
def decay(y_0, y_N, k):
    def y(x):
        return y_N - (y_N - y_0) * np.exp(- k * x)
    return y

In [ ]:
N = 10

L_0 = 50 * 0.3
L_N = 50
k_L =  - np.log(0.05) / N
L = decay(L_0, L_N, k_L)

r_0 = 0.5
r_N = 0.1
k_r =  - np.log(0.05) / N
r = decay(r_0, r_N, k_r)

theta_0 = 0
theta_N = 45
k_theta = - np.log(0.05) / N
theta = decay(0, 45, k_theta)

r1 = r_0
C_list = []
L_list = []

start_point = [0, 0, 0]
orientation = [0, 0, 0]

Theta = [orientation[1]]
Phi = [orientation[2]]
X = [start_point[0]]
Y = [start_point[1]]
Z = [start_point[2]]


for i in range(N):
    l = L(i) - sum(L_list)
    r2 = r(i)
    t = 5 - 5 * np.random.rand()
    phi = 5 - 5 * np.random.rand()
    z = l * np.cos(np.deg2rad(np.sum(Theta) + t))
    w = l * np.sin(np.deg2rad(np.sum(Theta) + t))
    x = w * np.cos(np.deg2rad(np.sum(Phi) + phi))
    y = w * np.sin(np.deg2rad(np.sum(Phi) + phi))
    
    print(f'L: {l:.03f} \t r: {r1:.03f} , {r2:.03f} \t t: {t:.03f}')
    
    
    c = translate([sum(X), sum(Y), sum(Z)])(
        rotate([0, sum(Theta) + t, sum(Phi) + phi])(
            cylinder(h=l, r1=r1, r2=r2)
        )
    )
        
    
    C_list.append(c.copy())
    L_list.append(l)
    r1 = r2
    
    Theta.append(t)
    Phi.append(phi)
    X.append(x)
    Y.append(y)
    Z.append(z)

In [ ]:
renderer.render(union()(*C_list))